In [1]:
import pandas as pd
import numpy as np

test_label_np = np.load('test_label_np.npy')
test_label = pd.read_csv("test_label.csv")

In [12]:
print(test_label_np.shape)
print(test_label.shape)

print(test_label.head())
print(test_label_np[:5])

(25000, 4375)
(25000, 3)
   Unnamed: 0                                  filename  \
0           0  00052fbb-bc66-4981-a5b8-657cece65474.jpg   
1           1  00056100-079c-4340-bf3b-fa0c39672388.jpg   
2           2  00066881-c3d3-42e7-8574-f4837dfa8e96.jpg   
3           3  0008378f-7e9e-49d5-b54d-04a271647aa9.jpg   
4           4  000908b5-6f73-4582-a79f-2de5e7094800.jpg   

                                              vector  
0  [0.12004861, -0.76521045, 0.15136011, 0.099144...  
1  [0.92215645, -0.9495212, 0.14619908, -0.628865...  
2  [0.59581876, 0.012878652, 1.0706635, 0.0139149...  
3  [0.42760938, -0.6176873, -0.07277152, 0.072084...  
4  [0.24775293, -0.3519494, -0.2148024, 0.1405869...  
[[-3.2140656 -3.6972532 -4.6058135 ... -3.1882322 -2.7203913 -4.542957 ]
 [-3.1019309 -2.1691084 -4.0610914 ... -2.921446  -3.6393511 -4.4575143]
 [-3.527786  -3.9894946 -4.693589  ... -3.1319866 -2.7486699 -4.549728 ]
 [-3.886122  -2.6658309 -4.3919845 ... -3.925885  -3.9227502 -4.450263 

In [2]:
test_query = pd.read_csv("data/test_query.csv", header=None)
test_query = test_query[0].map(lambda x: eval(x))
print(test_query.head())

0    [双手, 草坪, 球杆, 上挥, 墨镜, 帽子, 男士, 男人, 草地]
1            [东西, 夹子, 双手, 男人, 栈道, 女人, 道路]
2      [双手, 男人, 体育场, 草地, 双臂, 运动场, 穿着, 行走]
3                    [草坪, 女人, 红色, 墨镜, 帽子]
4        [穿着, 球衣, 球场上, 足球场, 运动场, 放在, 绿油油]
Name: 0, dtype: object


In [3]:
test_query_np = np.load("query_label_processed_np.npy")
test_query_sample = np.where(test_query_np[-1] == 1)
print(test_query_sample)

(array([   8,  682, 1887, 2598, 2662, 3186, 3423], dtype=int64),)


In [5]:
# test_query_np = []
# for i in test_query:
#     arr = np.zeros([4375])
#     for j in i:
#         if j in cate:
#             arr[cate[j]] = 1
#     test_query_np.append(arr)
#
# test_query_np = np.array(test_query_np)
# np.save("query_label_np.npy", test_query_np)

In [4]:
import json
with open("category_map.json", "r") as f:
    cate = json.load(f)

r_cate = {cate[i]: i for i in cate}

for i in test_query_sample[0]:
    print(i, r_cate[i])

8 衣着
682 身穿
1887 条纹
2598 屋子里
2662 眼镜
3186 男人
3423 手臂


In [17]:
from dataset.cocodataset import CoCoDataset

train_ds = CoCoDataset("", "", input_transform=None, labels_path="query2labels/data/coco/train_label_vectors.npy")
for i in np.where(train_ds[0][1] == 1)[0]:
    print(i, r_cate[i])

945 舞台
2518 地站
2721 话筒
3186 男人


In [5]:
# 计算余弦相似度
def cosine_similarity(vec1, vec2):
    dot_product = np.dot(vec1, vec2)
    norm1 = np.linalg.norm(vec1)
    norm2 = np.linalg.norm(vec2)
    similarity = dot_product / (norm1 * norm2)
    return similarity

In [25]:
def label_filter(query, label_vec):
    query = np.where(query == 1, True, False)
    return np.sum(label_vec[query])

In [6]:
from tqdm import tqdm

with open("submit_v9.csv", "w") as f:
    # 对于每个查询，计算相似度并选择top_k图片
    for query_vec in tqdm(test_query_np):
        similarities = np.array([cosine_similarity(query_vec, img_vec) for img_vec in test_label_np])
        top_k_indices = np.argsort(-similarities)[:5]  # 假设选择相似度最高的5张图片
        line = ','.join(list(test_label['filename'][top_k_indices])) + '\n'
        f.write(line)

 39%|███▉      | 9804/25000 [1:01:50<1:38:50,  2.56it/s]C:\Users\23518\AppData\Local\Temp\ipykernel_19604\2719652357.py:6: RuntimeWarning: invalid value encountered in double_scalars
  similarity = dot_product / (norm1 * norm2)
 40%|████      | 10068/25000 [1:03:34<2:36:37,  1.59it/s]C:\Users\23518\AppData\Local\Temp\ipykernel_19604\2719652357.py:6: RuntimeWarning: invalid value encountered in double_scalars
  similarity = dot_product / (norm1 * norm2)
 67%|██████▋   | 16824/25000 [1:47:35<51:55,  2.62it/s]  C:\Users\23518\AppData\Local\Temp\ipykernel_19604\2719652357.py:6: RuntimeWarning: invalid value encountered in double_scalars
  similarity = dot_product / (norm1 * norm2)
100%|██████████| 25000/25000 [2:39:22<00:00,  2.61it/s]
